# livedoorニュース分類 BERT版
---
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kyo46n/AI-workbook/blob/master/LivedoorNewsClassification_BERT_Pytorch.ipynb)

## 必要ライブラリのインストール

In [ ]:
!pip install transformers["ja"]

     |████████████████████████████████| 1.5MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 28.3MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
     |████████████████████████████████| 13.4MB 250kB/s 
     |████████████████████████████████| 481kB 53.7MB/s 
     |████████████████████████████████| 47.3MB 136kB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b3d4e6282c4cc158bf8a9bd31aecdc5b9e77166de676dbea69095c0ba9853b72
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for ipadic: filename=ipadic-1.0.0-cp36-none-any.whl size=13556727 sha256=53f2153412921caedc68645233008fb020686d65c460850d04ab2a10107d7381
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
  Created wheel for unidic: filename=unidic-1.0.3-cp36-none-any.whl size=5498 sha256=85d5697fedbae397e980853799c0eda0be83b18e4f191d81b23

## 前処理

In [ ]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split

# Livedoorニュースのファイルをダウンロードして解凍
!curl -O "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
!tar xf ldcc-20140209.tar.gz
print("")

# カテゴリーのフォルダのみを抽出
path = "./text/"
dirs = [name for name in os.listdir(path) if os.path.isdir(path+name)]
print("カテゴリー数:", len(dirs))
print(dirs)
print("")

# 各フォルダ内の各ファイル(LICENSE.txt以外)の本文(3行目以降)を抽出・整形し、各カテゴリをインデックス番号として二重リストに格納
text_label_data = []  # 文章とラベルのセット
dir_count = 0  # ディレクトリ数のカウント
file_count= 0  # ファイル数のカウント
for i in range(len(dirs)):
    dir = dirs[i]
    files = glob.glob(path + dir + "/*.txt")  # ファイルの一覧
    dir_count += 1
    for file in files:
        if os.path.basename(file) == "LICENSE.txt":
            continue
        with open(file, "r") as f:
            text = f.readlines()[3:]
            text = "".join(text)
            text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""})) 
            text_label_data.append([text, i])
        file_count += 1
        print("\rfiles: " + str(file_count) + ", dirs: " + str(dir_count), end="")

# 訓練用、検証用、テスト用に分割
train_val, test =  train_test_split(text_label_data, shuffle=True)
train, val =  train_test_split(train_val, shuffle=False)

# 形状確認
df_train = pd.DataFrame(train)
df_val = pd.DataFrame(val)
df_test = pd.DataFrame(test)
print("")
print(len(df_train))
print(len(df_val))
print(len(df_test))
display(df_train.head())

# csv保存
df_train.to_csv("news_train.csv", header=False, index=False)
df_val.to_csv("news_val.csv", header=False, index=False)
df_test.to_csv("news_test.csv", header=False, index=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8647k  100 8647k    0     0  3566k      0  0:00:02  0:00:02 --:--:-- 3564k

カテゴリー数: 9
['sports-watch', 'peachy', 'smax', 'topic-news', 'it-life-hack', 'dokujo-tsushin', 'livedoor-homme', 'movie-enter', 'kaden-channel']
files: 7367, dirs: 94143
1382
1842


,0,1
0,Excelで連番を入力したい場合、オートフィルの機能を使うと、マウスのドラッグだけで簡単に連...,4
1,先日公開されたlivedoorニュースのリーダーアプリ「三面貴族 by livedoorニュ...,4
2,今週末は『ダークナイト ライジング』が公開するなど、夏休みシーズンで盛り上がりを見せる映画興...,7
3,【前回までのあらすじ】大富豪から一転して貧乏になった勅使河原 栄華（てしがわら えいか）は、...,7
4,ビジネス、レジャー、デート・・・。あらゆるシーンで重宝されるゴルフは、もはやデキルオトコの“...,6


## 形態素解析・ベクトル化・DataLoader作成

- transformersのライブラリからBERT用の日本語Tokenizerを呼び出す。
- torchtextを使用して形態素解析・[CLS]と[SEP]の付与・マスク・単語ID化を行い、datasetとして保持する。
- datasetをdataloaderに入れてミニバッチ分割する。

In [ ]:
from transformers import BertJapaneseTokenizer
import torchtext

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
def tokenizer_512(input_text):
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt', truncation=True)[0]

max_length = 512
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

dataset_train, dataset_val, dataset_test = torchtext.data.TabularDataset.splits(
    path="./", train="news_train.csv", validation="news_val.csv",
    test="news_test.csv", format="csv", fields=[("Text", TEXT), ("Label", LABEL)])

In [ ]:
# datasetの確認
print("trainデータ数 :", dataset_train.__len__())
print("valデータ数 :", dataset_val.__len__())
print("testデータ数 :", dataset_test.__len__())
print("")
print("データ例 :")
item = next(iter(dataset_train))
print("形態素ID :", item.Text.tolist())
print("文章 :", tokenizer.convert_ids_to_tokens(item.Text))
print("長さ：", len(item.Text))  # [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベルindex：", int(item.Label))
print("ラベル：", dirs[int(item.Label)])

trainデータ数 : 4143
valデータ数 : 1382
testデータ数 : 1842

データ例 :
形態素ID : [2, 6927, 2953, 28595, 12, 195, 246, 11, 5965, 15, 1549, 344, 6, 4438, 7429, 5, 1197, 11, 3002, 13, 6, 10843, 5, 14097, 687, 12, 5880, 7, 1557, 15, 10, 1676, 11, 5965, 34, 146, 14, 392, 8, 1142, 6, 12959, 5, 1676, 11, 9827, 5965, 15, 1549, 344, 6, 14097, 15, 16, 3085, 11, 1424, 34, 5, 28, 8562, 75, 8, 26590, 344, 9, 7406, 11, 196, 28525, 205, 8, 25035, 4021, 489, 7446, 143, 1069, 28509, 7429, 1197, 11, 666, 15, 16, 195, 246, 11, 5965, 34, 4438, 7429, 12, 9, 6, 1557, 1676, 11, 5965, 15, 1549, 3085, 11, 14097, 12, 1424, 34, 727, 14, 31, 8, 3242, 5, 1676, 11, 5965, 15, 1549, 344, 6, 4180, 11, 18275, 15, 895, 3085, 11, 1424, 34, 5, 9, 8562, 75, 15, 6, 3219, 2577, 126, 1424, 3171, 312, 2575, 5, 29, 14604, 20496, 8, 7406, 5, 7429, 11, 666, 3171, 312, 6, 5965, 34, 1676, 5, 2194, 13, 6, 615, 28593, 29368, 6, 1140, 5, 1849, 11, 1374, 34, 45, 12, 6, 4645, 18, 276, 687, 195, 246, 11, 5965, 34, 146, 14, 392, 8, 19305, 1557, 1676, 5, 5

In [ ]:
# DataLoader(iterator)の作成
batch_size = 8
dl_train = torchtext.data.Iterator(dataset_train, batch_size=batch_size, train=True)
dl_val = torchtext.data.Iterator(dataset_val, batch_size=batch_size, train=False, sort=False)
dl_test = torchtext.data.Iterator(dataset_test, batch_size=batch_size, train=False, sort=False)
dataloaders_dict = {"train": dl_train, "val": dl_val}

In [ ]:
# DataLoaderの確認
batch = next(iter(dl_test)) # 最初のミニバッチ
print(batch)
print("")
print(batch.Text[0]) # ID+paddingされたベクトル
print(batch.Text[1]) # padding前の文の長さ
print("")
print(batch.Label) # 正解ラベル


[torchtext.data.batch.Batch of size 8]
	[.Text]:('[torch.LongTensor of size 8x512]', '[torch.LongTensor of size 8]')
	[.Label]:[torch.LongTensor of size 8]

tensor([[    2,  1133,     7,  ...,  2992,    54,     3],
        [    2,  2738,    12,  ..., 26197, 28566,     3],
        [    2,    36,  1953,  ...,  1964,    13,     3],
        ...,
        [    2,    91,    12,  ..., 30203,    35,     3],
        [    2,   101,    37,  ...,    65,    13,     3],
        [    2,  1216,  1617,  ...,     0,     0,     0]])
tensor([512, 512, 512, 512, 512, 512, 512, 447])

tensor([5, 7, 5, 5, 7, 8, 5, 6])


## モデルの設定
- 日本語学習済みBERTモデルもtransformersライブラリから使用。
- BERT最終層の後に全結合層を追加し、最終層sequence_outputの[CLS]部分が持つ768次元の特徴量が文章全体の特徴を表すものとして9クラスに分類する。
- ファインチューニングの練習のためBERT最終層のパラメータも変更する。
- optimizerにadamを使用し、学習率は全結合層の方を高めに設定する。
- クラス分類のためクロスエントロピー誤差を損失関数として使用する。

In [ ]:
from transformers import BertModel
from torch import nn
import torch.optim as optim

# 日本語学習済みBERTモデル
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# BERTに9クラス判定層をつなげたモデル
class BertForLivedoor(nn.Module):
    def __init__(self):
        super(BertForLivedoor, self).__init__()
        self.bert = model 
        self.cls = nn.Linear(in_features=768, out_features=9) # 層追加。入力はBERTの出力特徴量の次元768、出力は9クラス

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input):
        result = self.bert(input, output_attentions=True)
        # input： batch.Text[0]
        # result: (sequence_output, pooled_output, attentions)

        # sequence_outputの先頭のベクトルを抜き出す
        vec = result[0][:, 0, :]  # sequence_outputから[全バッチ, 先頭0番目の単語([CLS]), 全768要素]取り出し
        vec = vec.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換→この行不要な気もする
        output = self.cls(vec) # 全結合層

        attentions = result[2] # 可視化用

        return output, attentions

# インスタンス作成
net = BertForLivedoor()

# 勾配計算を最後のBertLayerと追加した全結合層のみに設定 (全ての勾配計算をFalseにした後、後ろ二層をTrueに変更)
for param in net.parameters():
    param.requires_grad = False
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True
for param in net.cls.parameters():
    param.requires_grad = True

# 最適化手法
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}])

# 損失関数
criterion = nn.CrossEntropyLoss()

## 学習・検証・評価

In [ ]:
import torch

# 学習させる関数
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    net.to(device)
    torch.backends.cudnn.benchmark = True # ネットワークがある程度固定であれば、高速化させる
    batch_size = dataloaders_dict["train"].batch_size # ミニバッチのサイズ

    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル
                optimizer.zero_grad()
                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs, _ = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if (iteration % 100 == 0):  # 100iterに1度lossを表示
                            acc = (torch.sum(preds == labels.data)).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 100iter. || 本イテレーションの正解率：{}'.format(iteration, loss.item(),  acc))
                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))

    return net

In [ ]:
# 学習・検証の実行
num_epochs = 5
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-----start-------
イテレーション 100 || Loss: 0.8118 || 100iter. || 本イテレーションの正解率：0.5
イテレーション 200 || Loss: 0.3062 || 100iter. || 本イテレーションの正解率：1.0
イテレーション 300 || Loss: 0.4890 || 100iter. || 本イテレーションの正解率：0.875
イテレーション 400 || Loss: 0.5919 || 100iter. || 本イテレーションの正解率：0.875
イテレーション 500 || Loss: 0.1486 || 100iter. || 本イテレーションの正解率：1.0
Epoch 1/5 | train |  Loss: 0.7125 Acc: 0.7642
Epoch 1/5 |  val  |  Loss: 0.3877 Acc: 0.8741
イテレーション 100 || Loss: 0.4710 || 100iter. || 本イテレーションの正解率：0.875
イテレーション 200 || Loss: 0.1387 || 100iter. || 本イテレーションの正解率：1.0
イテレーション 300 || Loss: 0.2087 || 100iter. || 本イテレーションの正解率：0.875
イテレーション 400 || Loss: 0.1448 || 100iter. || 本イテレーションの正解率：1.0
イテレーション 500 || Loss: 0.1496 || 100iter. || 本イテレーションの正解率：1.0
Epoch 2/5 | train |  Loss: 0.2767 Acc: 0.9117
Epoch 2/5 |  val  |  Loss: 0.3397 Acc: 0.8965
イテレーション 100 || Loss: 0.1232 || 100iter. || 本イテレーションの正解率：1.0
イテレーション 200 || Loss: 0.0230 || 100iter. || 本イテレーションの正解率：1.0
イテレーション 300 || Loss: 0.0258 || 100iter. || 本イテレーションの正解率

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_trained.eval()
net_trained.to(device)

# epochの正解数を記録する変数
epoch_corrects = 0
for batch in tqdm(dl_test):
    inputs = batch.Text[0].to(device)  # 単語IDベクトル
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):
        outputs, _ = net_trained(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)
print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))

100%|██████████| 231/231 [01:10<00:00,  3.28it/s]

テストデータ1842個での正解率：0.9311


## Attentionの可視化
BERT最終層のmulti-head attention 12個の合計を可視化

In [ ]:
# テストデータのうち少しだけ可視化してみる
batch_size = 4
dl_viz = torchtext.data.Iterator(dataset_test, batch_size=batch_size, train=False, sort=False)

batch = next(iter(dl_viz))
inputs = batch.Text[0].to(device)
labels = batch.Label.to(device)

outputs, attentions = net_trained(inputs)
_, preds = torch.max(outputs, 1) # ラベル予測

# 最終層のattentionのsize確認 (batch_size, num_heads, sequence_length, sequence_length)
print(attentions[-1].size())

torch.Size([4, 12, 512, 512])


In [ ]:
from IPython.display import HTML

# Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力
def highlight(word, attn):
    html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)

# HTMLを作成
def mk_html(index, batch, preds, attentions):
    # index : 個別データのインデックス番号, dirs : カテゴリーのリスト
    sentence = batch.Text[0][index]
    label = batch.Label[index]
    pred = preds[index]
    label_str = dirs[label]
    pred_str = dirs[pred]
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    html += '[BERT最終層のAttention合計]<br>'

    # 文章の長さ分のzero tensorを宣言
    seq_len = attentions.size()[2]
    all_attens = torch.zeros(seq_len).to(device)

    # Attention12個を足し算
    for i in range(12):
      all_attens += attentions[index, i, 0, :] # 0は[CLS]

    for word, attn in zip(sentence, all_attens):
      # 単語が[SEP]の場合は文章が終わりなのでbreak
      if tokenizer.convert_ids_to_tokens([word.tolist()])[0] == "[SEP]":
        break
      # IDを単語に戻してhighlight関数で着色
      html += highlight(tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0], attn)

    html += "<br><br>"

    return html

In [ ]:
# 正答の例
index = 2
html_output = mk_html(index, batch, preds, attentions[-1])
HTML(html_output)

In [ ]:
# 誤答の例
index = 1
html_output = mk_html(index, batch, preds, attentions[-1])
HTML(html_output)

## 感想・考察
- ネット検索ではMeCabなど形態素解析ツール群のインストールがごちゃごちゃしていたが、huggingface社の公式サイトに日本語版向けのガイド(pip install transformers["ja"])があって楽だった。
- BERTのモデルもトークナイザーもtransformersライブラリが充実していてとても助かった。
- 学習と検証のスコアを見ると2エポック目からすでに正答率9割を超えており、性能の高さが伺える。テストデータでも正答率90%程度となり、分類は十分機能していると思われる。
- 正答例の可視化結果をみると、「ブログ」「女」「女の子」などが着色されており、独女通信に特徴的な単語が認識されている様子だった。しかし「独女」や「乙女」など、もっと明確に雑誌の特徴を表現しそうな単語は色づいておらず、単語の頻出度や一般性も加味されているように思う。
- 誤答例の可視化結果では、「スマートフォン」「TV」「機能」などが着色されており、映画に関するキーワードが文章中にほとんどないことから誤答したものと思われる。実際どちらの雑誌に乗っていても不思議ではない記事と感じた。

## 参考
- https://huggingface.co/transformers/
- https://qiita.com/sugulu_Ogawa_ISID/items/697bd03499c1de9cf082
- https://lab.m-field.co.jp/2020/11/30/search-by-bert/
- https://qiita.com/m__k/items/e312ddcf9a3d0ea64d72
- BERTによる自然言語処理を学ぼう！ -Attention、TransformerからBERTへとつながるNLP技術- (Udemy)
- つくりながら学ぶPytorchによる発展ディープラーニング 小川雄太郎 (書籍)